![alt text](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTK4gQ9nhwHHaSXMHpeggWg7twwMCgb877smkRmtkmDeDoGF9Z6&usqp=CAU)
# <font color='Blue'>Ciência dos Dados na Prática</font>

# <font color='purple'> Prevendo Efeitos Colaterais de Vacinas - Inteligência Artificial </font>

![](https://www.news-medical.net/image-handler/picture/2021/4/WIW_Banner_Facebook_Format-6.gif)

![](https://stories.cnnbrasil.com.br/wp-content/uploads/sites/9/2021/07/vacina-2.gif)

# 1° Problema de Negócio

### Precisamos criar um Sistema de Previsão de Efeitos Colaterais das Vacinas COVID 19, baseado em Inteligência Artificial.


# 2° Análise Exploratória 

In [ ]:
!wget "https://github.com/XxAnirudh-RameshxX/Vaccine-side-effect-detection/raw/main/VAERS_processed.csv"

In [3]:
import pandas as pd
import gzip
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

In [4]:
!wget "https://snap.stanford.edu/biodata/datasets/10021/files/D-DoMiner_miner-diseaseDOID.tsv.gz"
with gzip.open("D-DoMiner_miner-diseaseDOID.tsv.gz") as f:
    df = pd.read_csv(f, sep='\t')
l=list(df["Name"])
len(l)

--2021-11-05 18:08:33--  https://snap.stanford.edu/biodata/datasets/10021/files/D-DoMiner_miner-diseaseDOID.tsv.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 430481 (420K) [application/x-gzip]
Saving to: ‘D-DoMiner_miner-diseaseDOID.tsv.gz’

D-DoMiner_miner-dis 100%[===================>] 420.39K  1.41MB/s    in 0.3s    

2021-11-05 18:08:34 (1.41 MB/s) - ‘D-DoMiner_miner-diseaseDOID.tsv.gz’ saved [430481/430481]



9247

In [ ]:
data = pd.read_csv("/content/VAERS_processed.csv")
data.head()

In [6]:
# required_cols = [3, 6, 24, 25, 26, 34, 42]
data = data.fillna(0)
data = data.replace(to_replace="Y", value=1)
data = data.replace(to_replace=False, value=0)
data = data.replace(to_replace=True, value=1)

In [ ]:
columns = data.columns
col_indices = {}
for i in range(len(columns)):
  col_indices[i] = columns[i]
col_indices

# 3° Pré-Processamento dos Dados

In [8]:
one_hot_history = [[0 for _ in range(10)] for _ in range(len(data))]
# len(data)
classes = [['blood pressure', 'htn', 'hypertension'], ['asthma'], ['diabetes'],
           ['covid-19', 'covid 19', 'covid', 'corona', 'sars'], ['hypothyroidism'],
           ['hyperlipidemia'], ['cholesterol'], ['depression'], ['obesity', 'obese', 'overweight', 'fat'],
           ['kidney']]
columns = ['Hypertension', 'Asthma', 'Diabetes', 'Corona', 'Hypothyroidism', 
           'Hyperlipidemia', 'Chelestrol', 'Depression', 'Obesity', 'Kidney']
history = data.iloc[:, 26:27].values.tolist()
for i in range(len(history)):
  if type(history[i][0]) == int:
    continue
  str_hist = history[i][0].lower()
  for j in range(len(classes)):
    for k in classes[j]:
      if k in str_hist:
        one_hot_history[i][j] = 1
        break
histroy_dataframe = pd.DataFrame(one_hot_history, columns=columns)
print(histroy_dataframe)
print(len(histroy_dataframe))

      Hypertension  Asthma  Diabetes  ...  Depression  Obesity  Kidney
0                0       0         0  ...           0        0       0
1                0       0         0  ...           0        0       0
2                0       0         0  ...           0        0       0
3                1       0         0  ...           0        0       0
4                0       1         0  ...           0        0       0
...            ...     ...       ...  ...         ...      ...     ...
5274             0       0         0  ...           0        0       0
5275             0       0         0  ...           0        0       0
5276             0       0         0  ...           0        0       0
5277             0       0         0  ...           0        0       0
5278             0       0         0  ...           0        0       0

[5279 rows x 10 columns]
5279


In [9]:
required_cols = [3, 6 , 42] + [i for i in range(82, 92)]
additional_labels = ["Pain", 'Pyrexia', 'Headache', 'Dyspnoea', 'Fatigue', 'Chills', 'Dizziness', 'Nausea', 'Asthenia', 'Cough']
# additional_labels = [43]
data = pd.concat([data, histroy_dataframe], axis=1)
columns = data.columns.values
required_cols = columns[required_cols]
X = data[required_cols].iloc[:, :]
y = data[additional_labels].iloc[:, :]
oe = OrdinalEncoder()
oe.fit(X[["VAX_NAME"]])
X["VAX_NAME"] = oe.transform(X[["VAX_NAME"]])
oe_sex = OrdinalEncoder()
oe_sex.fit(X[["SEX"]])
X["SEX"] = oe_sex.transform(X[["SEX"]])
# print(X.shape)
# print(y.shape)
# print(X)
# print(y)
# print(X.columns)
# data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [11]:
print(X_train)
print(y_train)

      AGE_YRS  SEX  VAX_NAME  ...  Depression  Obesity  Kidney
4944     39.0  0.0       0.0  ...           0        0       0
2835     26.0  0.0       1.0  ...           0        0       0
2117      0.0  2.0       1.0  ...           0        0       0
1650     57.0  1.0       1.0  ...           0        0       0
893      36.0  0.0       1.0  ...           0        0       0
...       ...  ...       ...  ...         ...      ...     ...
3092     92.0  1.0       1.0  ...           0        0       0
3772      0.0  2.0       1.0  ...           0        0       0
5191     92.0  1.0       1.0  ...           0        0       0
5226     45.0  0.0       0.0  ...           0        0       0
860      83.0  0.0       1.0  ...           0        0       0

[4751 rows x 13 columns]
      Pain  Pyrexia  Headache  Dyspnoea  ...  Dizziness  Nausea  Asthenia  Cough
4944   1.0      1.0       0.0       0.0  ...        0.0     0.0       0.0    0.0
2835   0.0      1.0       0.0       0.0  ...        1.0 

# 4° Criando Máquinas Preditivas

In [12]:
from keras.callbacks import ModelCheckpoint
model_checkpoint = ModelCheckpoint('MP.h5', monitor="val_accuracy", save_best_only=True)

In [13]:
model = Sequential()
model.add(Dense(64, input_shape=X_train.shape[1:], activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.1))
model.add(Dense(10, activation="sigmoid"))
model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                896       
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               3

In [15]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[model_checkpoint])

Epoch 1/100
149/149 [==============================] - 2s 5ms/step - loss: 0.3783 - accuracy: 0.4111 - val_loss: 0.3355 - val_accuracy: 0.7519
Epoch 2/100
149/149 [==============================] - 1s 4ms/step - loss: 0.3411 - accuracy: 0.6060 - val_loss: 0.3336 - val_accuracy: 0.7519
Epoch 3/100
149/149 [==============================] - 1s 4ms/step - loss: 0.3348 - accuracy: 0.6622 - val_loss: 0.3383 - val_accuracy: 0.7519
Epoch 4/100
149/149 [==============================] - 1s 4ms/step - loss: 0.3322 - accuracy: 0.7125 - val_loss: 0.3310 - val_accuracy: 0.7519
Epoch 5/100
149/149 [==============================] - 1s 4ms/step - loss: 0.3297 - accuracy: 0.7083 - val_loss: 0.3302 - val_accuracy: 0.7519
Epoch 6/100
149/149 [==============================] - 1s 4ms/step - loss: 0.3291 - accuracy: 0.7104 - val_loss: 0.3291 - val_accuracy: 0.7519
Epoch 7/100
149/149 [==============================] - 1s 4ms/step - loss: 0.3283 - accuracy: 0.7230 - val_loss: 0.3304 - val_accuracy: 0.7519

In [16]:
from keras.models import load_model
model1 = load_model("MP.h5")

In [ ]:
model1.evaluate(X_test, y_test)

In [18]:
x = X_train.iloc[:1]

In [ ]:
x

In [20]:
x = np.array(x)

In [21]:
x

array([[39.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [ ]:
x.shape

In [23]:
y = model1.predict(x)

In [ ]:
y

In [25]:
symptoms_label = y_train.columns

In [ ]:
symptoms_label

In [ ]:
symptoms_label[1]

In [ ]:
indices = (-y).argsort()[:3]
indices

In [ ]:
y.shape

In [ ]:
values = []
labels = []
for count in range(3):
  idx = np.where(indices==count)
  print(len(idx))
  values.append(y[0][idx[1][0]])
  labels.append(symptoms_label[idx[1][0]])


In [ ]:
values
labels

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.show()

In [ ]:
X_train.columns

# 1° Problema de Negócio

# 2° Análise Exploratória 

# 3° Pré-Processamento dos Dados

# 4° Criando Máquinas Preditivas

#Fim

## Valeu!

### # Onde me Encontrar - Ciência dos Dados <a href="https://linktr.ee/cienciadosdados">https://linktr.ee/cienciadosdados</a>

In [ ]:
from IPython.core.display import HTML
HTML('<iframe width="320" height="180" src="https://www.youtube.com/embed/WH6_pWe2_r0" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

* Fontes Úteis:

https://github.com/murali22chan/Vaccine.AI

https://vaccine-ai.herokuapp.com/
